In [2]:
from collections import Counter
import numpy as np
import trino
import pandas as pd
##mysql connector
import mysql.connector
myconn = mysql.connector.connect(host = "tango-read-nifi.ci9kqv9mlekc.ap-south-1.rds.amazonaws.com",
                                 user = "prakhar",passwd = "otipy@1234",database = "consumer")
cur_mysql = myconn.cursor()


##trino connector
from trino.dbapi import connect
conn = connect(host="presto.crofarm.com",port=443,catalog='hive',http_scheme='https',auth=trino.auth.BasicAuthentication("db.manager","crofarm"))
cur_trino = conn.cursor()


trino_final_tables=[]
schema=["consumer","crofarmusers","products"]
for i in range(len(schema)):
    b="show tables from "+schema[i]
    cur_trino.execute(b)
    trino_tables=cur_trino.fetchall()
    for j in range(len(trino_tables)):
        trino_final_tables.append(str(schema[i].lower())+"."+str(trino_tables[j][0].lower()))

trino_final_tables.sort()

mysql_final_tables=[]
schema=["consumer","crofarmUsers","products"]
for i in range(len(schema)):
    b="show tables from "+schema[i]
    cur_mysql.execute(b)
    mysql_tables=cur_mysql.fetchall()
    for j in range(len(mysql_tables)):
        mysql_final_tables.append(str(schema[i].lower())+"."+str(mysql_tables[j][0].lower()))

mysql_final_tables.sort()

    
not_present_in_trino=list(set(mysql_final_tables)-set(trino_final_tables))
new_list=list(set(mysql_final_tables)-set(not_present_in_trino))
new_list.sort()
trino_final_tables.sort()

columns=[]
columns2=[]
for i in range(len(new_list)):
    if(new_list[i]==trino_final_tables[i]):
        cur_mysql.execute("select COLUMN_NAME from information_schema.columns where `TABLE_SCHEMA`='"+new_list[i].split('.')[0]+"' and table_name ='"+new_list[i].split('.')[1]+"'")
        columns.append(cur_mysql.fetchall())
        cur_trino.execute("select COLUMN_NAME from information_schema.columns where TABLE_SCHEMA='"+trino_final_tables[i].split('.')[0]+"' and table_name ='"+trino_final_tables[i].split('.')[1]+"'")
        columns2.append(cur_trino.fetchall())

col=[]
for i in range(len(columns)):
    m=[]
    for j in range(len(columns[i])):
        m.append(columns[i][j][0])
    col.append(m)
    
col2=[]
for i in range(len(columns2)):
    m=[]
    for j in range(len(columns2[i])):
        m.append(columns2[i][j][0])
    col2.append(m)
    
change_table=[]
dif_col=[]
for i in range(len(col)):
    dif_col.append((list(set(col[i])-set(col2[i]))))
    a=(list(set(col[i])-set(col2[i])))
    if(len(a)>0):
        print(a)
        print(new_list[i])
        change_table.append(new_list[i])
        
diff_final=[]
for i in range(len(dif_col)):
    if(dif_col[i]==[]):
        pass
    else:
        diff_final.append(dif_col[i])

['bankRRN']
consumer.cn_source_refund
['BANKRRN', 'TxnInitDate', 'PayerVA', 'TxnCompletionDate']
consumer.icici_transaction
['transactionId', 'requestTime']
consumer.sodexo_transaction
['Fruit_closing_value', 'sec_Greens_opening_value', 'Exotic_margin_per', 'Exotic_margin', 'Fruit_opening_value', 'POT_margin', 'Exotic_opening_value', 'sec_POT_opening_value', 'sec_Exotic_opening_value', 'Greens_opening_value', 'sec_Exotic_closing_value', 'POT_margin_per', 'sec_Greens_closing_value', 'sec_Fruit_closing_value', 'Fruit_margin_per', 'POT_opening_value', 'Greens_margin', 'sec_POT_closing_value', 'Fruit_margin', 'Greens_closing_value', 'POT_closing_value', 'Greens_margin_per', 'sec_Fruit_opening_value', 'Exotic_closing_value']
products.dc_metric


In [255]:
print(len(change_table))
change_table

33


['consumer.cn_agent_first_order_performance',
 'consumer.cn_agent_performance',
 'consumer.cn_bonus_wallet_transaction',
 'consumer.cn_consumer_property',
 'consumer.cn_consumer_subscribed_products',
 'consumer.cn_issue_delivery',
 'consumer.cn_order_property',
 'consumer.cn_picker',
 'consumer.cn_source_refund',
 'consumer.cn_wallet_coupon',
 'consumer.delivery_rating',
 'consumer.icici_transaction',
 'consumer.order_data_property',
 'consumer.order_delivery_info',
 'consumer.partner_rejected_order',
 'consumer.product_rating',
 'consumer.route_line',
 'consumer.sodexo_transaction',
 'products.b2c_margin',
 'products.bunch_packing_qc_mapping',
 'products.bunch_table_packing',
 'products.consumer_first_order',
 'products.dc_metric',
 'products.dt_route',
 'products.dynamic_banners',
 'products.partner_incentive_sales',
 'products.product_sku_receiving_mapping',
 'products.qc_bunch_mapping',
 'products.sku_bunch',
 'products.sku_receiving',
 'products.zone_pricing',
 'products.zone_prod

In [281]:
change_table[0]

'consumer.cn_agent_first_order_performance'

In [494]:
('_').join(change_table[0].split('.'))

'consumer_cn_agent_first_order_performance'

In [438]:
len(diff_final)

33

In [3]:
f=[]
for k in range(len(change_table)):
    m=[]
    n=[]
    b={}
    cur_mysql.execute("desc "+str(change_table[k]))
    a=(cur_mysql.fetchall())
    for i in range(len(a)):
        b[(a[i][0])]=a[i][1]
    f.append(b)
typ=[]
for i in range(len(change_table)):
    m=[]
    for k in range(len(diff_final[i])):
        if('int(11)' == f[i][diff_final[i][k]]):
            m.append('int')
        elif('varchar' in f[i][diff_final[i][k]]):
            m.append('varchar')
        elif('longtext' in f[i][diff_final[i][k]]):
            m.append('varchar')
        elif('tinyint' in f[i][diff_final[i][k]]):
            m.append('boolean')
        elif('datetime' in f[i][diff_final[i][k]]):
            m.append('bigint')
        else:
            m.append(f[i][diff_final[i][k]])
    typ.append(m)

In [4]:
for i in range(len(diff_final)):
    print(len(typ[i])==len(diff_final[i]))

True
True
True
True


In [5]:
len(diff_final)

4

In [6]:
typ

[['varchar'],
 ['varchar', 'varchar', 'varchar', 'varchar'],
 ['varchar', 'int'],
 ['decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)']]

In [7]:
len(change_table)

4

## historical

In [497]:
for i in range(len(change_table)):
    for j in range(len(diff_final[i])):
#         cur_trino.execute
        print("ALTER TABLE IF EXISTS historical."+('_').join(change_table[i].split('.'))+" ADD COLUMN  IF NOT EXISTS  "+diff_final[i][j]+" "+typ[i][j])
#         print(cur_trino.fetchall())

ALTER TABLE IF EXISTS historical.consumer_cn_source_refund ADD COLUMN  IF NOT EXISTS  bankRRN int
ALTER TABLE IF EXISTS historical.consumer_icici_transaction ADD COLUMN  IF NOT EXISTS  BANKRRN int
ALTER TABLE IF EXISTS historical.consumer_icici_transaction ADD COLUMN  IF NOT EXISTS  TxnInitDate decimal(8,2)


IndexError: list index out of range

# live

In [ ]:
for i in range(len(change_table)):
    for j in range(len(diff_final[i])):
        print("ALTER TABLE IF EXISTS  "+change_table[i]+" ADD COLUMN  IF NOT EXISTS  "+diff_final[i][j]+" "+typ[i][j])
#         print(cur_trino.fetchall())

In [ ]:
ALTER TABLE IF EXISTS consumer.cn_bonus_wallet_transaction ADD COLUMN  IF NOT EXISTS  coupon_type VARCHAR

# Incremental_table want to change

In [455]:
cur_trino.execute("show tables from historical like 'inc%'")
final=cur_trino.fetchall()
full_load=[]
for i in range(len(final)):
    full_load.append(str(final[i][0]).split('_')[1:][0]+'.'+(str(('_').join(str(final[i][0]).split('_')[1:][1:]))))
t=set(change_table)-set(full_load)
final_inc=set(change_table)-set(t)
print(len(final_inc))
final_inc

5


{'consumer.cn_bonus_wallet_transaction',
 'consumer.cn_consumer_subscribed_products',
 'consumer.cn_issue_delivery',
 'consumer.cn_source_refund',
 'consumer.delivery_rating'}

# Update _table want to change

In [456]:
cur_trino.execute("show tables from historical like 'ut%'")
final=cur_trino.fetchall()
full_load=[]
for i in range(len(final)):
    full_load.append(str(final[i][0]).split('_')[1:][0]+'.'+(str(('_').join(str(final[i][0]).split('_')[1:][1:]))))
t=set(change_table)-set(full_load)
final_ut=set(change_table)-set(t)
print(len(final_ut))
final_ut

4


{'consumer.cn_order_property',
 'consumer.icici_transaction',
 'consumer.sodexo_transaction',
 'products.zone_product_deal'}

# full_load_table want to change

In [505]:
l=set(change_table)-set(final_ut)
full_table=set(l)-set(final_inc)
print(len(full_table))
full_table

1


{'products.dc_metric'}

In [507]:
a=['consumer.cn_agent_first_order_performance',
 'consumer.cn_agent_performance',
 'consumer.cn_bonus_wallet_transaction',
 'consumer.cn_consumer_property',
 'consumer.cn_consumer_subscribed_products',
 'consumer.cn_issue_delivery',
 'consumer.cn_order_property',
 'consumer.cn_picker',
 'consumer.cn_source_refund',
 'consumer.cn_wallet_coupon',
 'consumer.delivery_rating',
 'consumer.icici_transaction',
 'consumer.order_data_property',
 'consumer.order_delivery_info',
 'consumer.partner_rejected_order',
 'consumer.product_rating',
 'consumer.route_line',
 'consumer.sodexo_transaction',
 'products.b2c_margin',
 'products.bunch_packing_qc_mapping',
 'products.bunch_table_packing',
 'products.consumer_first_order',
 'products.dc_metric',
 'products.dt_route',
 'products.dynamic_banners',
 'products.partner_incentive_sales',
 'products.product_sku_receiving_mapping',
 'products.qc_bunch_mapping',
 'products.sku_bunch',
 'products.sku_receiving',
 'products.zone_pricing',
 'products.zone_product_deal',
 'products.zone_subscribable_product']

In [512]:
typ=[['int', 'decimal(8,2)'],
 ['int', 'decimal(8,2)'],
 ['varchar'],
 ['int', 'boolean', 'int', 'bigint', 'int', 'bigint'],
 ['int', 'int', 'int'],
 ['bigint', 'bigint'],
 ['bigint', 'bigint'],
 ['int'],
 ['varchar'],
 ['varchar', 'varchar'],
 ['int', 'int'],
 ['varchar', 'varchar', 'varchar', 'varchar'],
 ['bigint', 'bigint'],
 ['int'],
 ['varchar'],
 ['int'],
 ['int'],
 ['varchar', 'int'],
 ['decimal(6,3)'],
 ['int', 'varchar', 'varchar', 'varchar'],
 ['int',
  'varchar',
  'int',
  'varchar',
  'varchar',
  'varchar',
  'int',
  'varchar',
  'bigint',
  'decimal(8,2)',
  'bigint',
  'int',
  'int'],
 ['int'],
 ['decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)',
  'decimal(8,1)'],
 ['bigint', 'bigint'],
 ['int', 'varchar', 'int', 'int'],
 ['decimal(14,2)', 'decimal(14,2)'],
 ['boolean', 'boolean'],
 ['int', 'varchar', 'varchar', 'varchar', 'varchar', 'varchar', 'int'],
 ['varchar',
  'int',
  'varchar',
  'varchar',
  'int',
  'varchar',
  'varchar',
  'int',
  'int',
  'int'],
 ['varchar'],
 ['int', 'int'],
 ['int', 'int'],
 ['int', 'boolean']]

In [514]:
d=[['orders_delivered_by_six_thirty_am', 'six_thirty_am_delivery_per'],
 ['orders_delivered_by_six_thirty_am', 'six_thirty_am_delivery_per'],
 ['membership_end_slot',
  'welcome_bonus_expired',
  'membership_id',
  'created_at',
  'delivery_preference',
  'updated_at'],
 ['picker_type'],
 ['coupon_type', 'consumer_list'],
 ['updated_at', 'created_at'],
 ['time_diff'],
 ['action'],
 ['order_id'],
 ['line_type'],
 ['otp', 'otp_sent_time', 'email_status'],
 ['supplier_id', 'supplier_name', 'prod_name', 'image_id'],
 ['supplier_id',
  'crate_info',
  'machine_id',
  'prod_name',
  'child_prod_name',
  'image_id',
  'total_packed_crates',
  'supplier_name',
  'packing_resume_time',
  'child_in_kg',
  'packing_paused_time',
  'pause_status',
  'total_crates_assigned'],
 ['Fruit_closing_value',
  'sec_Greens_opening_value',
  'Exotic_margin_per',
  'Exotic_margin',
  'Fruit_opening_value',
  'POT_margin',
  'Exotic_opening_value',
  'sec_POT_opening_value',
  'sec_Exotic_opening_value',
  'Greens_opening_value',
  'sec_Exotic_closing_value',
  'POT_margin_per',
  'sec_Greens_closing_value',
  'sec_Fruit_closing_value',
  'Fruit_margin_per',
  'POT_opening_value',
  'Greens_margin',
  'sec_POT_closing_value',
  'Fruit_margin',
  'Greens_closing_value',
  'POT_closing_value',
  'Greens_margin_per',
  'sec_Fruit_opening_value',
  'Exotic_closing_value'],
 ['partnerbalance', 'consumer_reactivation'],
 ['is_cold', 'is_disassembler'],
 ['prod_name',
  'supplier_id',
  'qr_code',
  'bunch_type',
  'total_crates',
  'image_id',
  'supplier_name',
  'n_slot_id',
  'total_floor_crates',
  'total_cold_crates'],
 ['supplier_name'],
 ['substitute_prod_id', 'deactivated_at'],
 ['unit_price', 'monthly_pack']]

In [515]:
len(d)

20

In [516]:
len(typ)

33

In [511]:
'full_'+"_".join(a[0].split('.'))

'full_consumer_cn_agent_first_order_performance'

In [513]:
for i in range(len(change_table)):
    for j in range(len(diff_final[i])):
#         cur_trino.execute
        print("ALTER TABLE IF EXISTS historical.full_"+"_".join(a[0].split('.'))+" ADD COLUMN  IF NOT EXISTS  "+diff_final[i][j]+" "+typ[i][j])
#         print(cur_trino.fetchall())

ALTER TABLE IF EXISTS historical.full_consumer_cn_agent_first_order_performance ADD COLUMN  IF NOT EXISTS  bankRRN int
ALTER TABLE IF EXISTS historical.full_consumer_cn_agent_first_order_performance ADD COLUMN  IF NOT EXISTS  BANKRRN int
ALTER TABLE IF EXISTS historical.full_consumer_cn_agent_first_order_performance ADD COLUMN  IF NOT EXISTS  TxnInitDate decimal(8,2)


IndexError: list index out of range

In [245]:
len(change_table)==len(inc_table)+len(final_ut)+len(final_inc)

True

In [ ]:
ALTER TABLE IF EXISTS consumer.cn_bonus_wallet_transaction ADD COLUMN  IF NOT EXISTS  coupon_type VARCHAR